라이브러리 임포트

In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

데이터 로드 및 분리

In [3]:
data = "C:/GBAW14/Intro.txt"  # 파일 경로
with open(data, "r", encoding="utf-8") as d:
    text = d.read()

# 텍스트를 문장 단위로 분리
texts = text.strip().split("\n")
print(f"텍스트 분리 완료: {len(texts)}개의 문장")
print(texts)


텍스트 분리 완료: 8개의 문장
['디지털경영, 융합의 시대를 주도합니다.', '디지털경영전공은 다양한 배경을 가진 학생들이 모여 함께 성장하는 곳입니다. 전공과목은 프로그래밍과 데이터 분석에 필요한 기초 개념과 실습을 체계적으로 학습하도록 구성되어 있습니다. 때문에 문과 출신이더라도 열정과 노력으로 데이터 분석과 프로그래밍을 익힐 수 있습니다. 빅데이터와 인공지능의 기초를 익히고 기업 경영에 활용하는 방법을 익힌다면 여러분 앞에는 성공을 위한 무한한 가능성이 열릴 것입니다. 배우고자 하는 의지가 있고, 열정이 넘치는 모든 학생을 환영합니다.', '데이터 분석, 이제는 선택이 아니라 필수입니다.', '과거에 컴맹이라는 말이 있었다면, 이제는 "데이터맹"이라는 말을 듣는 시대가 올 것입니다. 기업이 책에 나오는 경영학 이론을 통해 의사결정을 하거나, 몇몇 담당자의 경험에 의존해 의사결정을 하는 시대는 끝났습니다. 기업의 모든 활동이 빅데이터로 축적되며 데이터를 효과적으로 수집, 분석, 활용하여 중요한 의사결정을 하는 Data driven decision making 시대입니다. 따라서, 경영학을 공부하는 학생들에게 데이터분석 능력은 기본 중의 기본이 되었습니다.', '빅데이터 인공지능과 경영학의 만남을 추구합니다.', '디지털경영전공은 데이터 분석 능력과 비즈니스 마인드를 겸비한 실무 융합형 인재 양성을 목표로 하고 있습니다. 빅데이터와 인공지능 기술을 마케팅, 금융, 인사조직, 생산운영관리 등 다양한 비즈니스 분야에 적용하여 기업의 성과 향상과 새로운 가치 창출에 기여합니다. 학생들은 머신러닝, 딥러닝, 비쥬얼라이제이션, 최적화 등의 최신 기술을 활용하여 데이터를 분석하고 인사이트를 도출하는 실습 경험을 통해 실무 역량을 강화할 수 있습니다.', '빅데이터 분석을 통해 가치를 창출하는 DATA ANALYST', '기업에서 필요한 가치는 인사이트를 통한 비즈니스 액션에 의해 창출됩니다. 빅데이터는 단순히 많이 모아 놓는다고, 기업에게 필요한 가치를 창출 할 수가

임베딩 모델 로드 및 문장 임베딩 생성

In [4]:
# SentenceTransformer 모델 로드
embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 문장 임베딩 생성
embeddings = embedding_model.encode(texts)

# FAISS 인덱스 생성
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product 기반 (코사인 유사도 계산)
index.add(np.array(embeddings))
print(f"FAISS 인덱스 생성 완료: {len(texts)}개의 벡터 추가")
print(embeddings)


FAISS 인덱스 생성 완료: 8개의 벡터 추가
[[-0.17000909  0.18103468 -0.05705997 ... -0.11454999  0.4161579
  -0.08160289]
 [ 0.11373063  0.16334301 -0.16447408 ... -0.06636404  0.09914879
  -0.08742449]
 [-0.12620933  0.27438313 -0.14733443 ...  0.08832772  0.29353812
  -0.03603112]
 ...
 [-0.05885859  0.18100345 -0.1944327  ... -0.09648705  0.15431458
  -0.05758542]
 [-0.18976808  0.10674233 -0.3034333  ... -0.17258222 -0.17873438
  -0.11350576]
 [-0.05061139  0.01395016 -0.23039877 ... -0.11627313  0.18779449
  -0.16345209]]


KoBERT 질문/답변 

In [5]:
model_name = "monologg/koelectra-base-v3-finetuned-korquad"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# 질문/답변 함수 
def answer_question_koelectra(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    input_ids = inputs["input_ids"]
    token_type_ids = inputs["token_type_ids"]

    with torch.no_grad():
        outputs = qa_model(input_ids=input_ids, token_type_ids=token_type_ids)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    start_idx = torch.argmax(start_scores)
    end_idx = torch.argmax(end_scores) + 1
    answer = tokenizer.decode(input_ids[0][start_idx:end_idx])
    return answer


검색 및 질문/답변

In [6]:
# 검색 함수 
def search(query, top_k=3):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]

# 질문/답변 함수 
def answer_question(query):
    retrieved_sentences = search(query, top_k=3)
    context = " ".join([sent[0] for sent in retrieved_sentences])
    
    # KoBERT QA 
    answer = answer_question_koelectra(query, context)
    return answer, retrieved_sentences

question = "디지털경영전공의 목표는 무엇인가요?"
answer, retrieved = answer_question(question)

print("질문:", question)
print("답변:", answer)
print("참조 문장:")
for sent, dist in retrieved:
    print(f"- {sent} (유사도: {dist:.2f})")

question = "기업에서 필요한 가치는 무엇인가요?"
answer, retrieved = answer_question(question)

print("질문:", question)
print("답변:", answer)
print("참조 문장:")
for sent, dist in retrieved:
    print(f"- {sent} (유사도: {dist:.2f})")

질문: 디지털경영전공의 목표는 무엇인가요?
답변: 실무 융합형 인재 양성
참조 문장:
- 디지털경영, 융합의 시대를 주도합니다. (유사도: 11.50)
- 디지털경영전공은 데이터 분석 능력과 비즈니스 마인드를 겸비한 실무 융합형 인재 양성을 목표로 하고 있습니다. 빅데이터와 인공지능 기술을 마케팅, 금융, 인사조직, 생산운영관리 등 다양한 비즈니스 분야에 적용하여 기업의 성과 향상과 새로운 가치 창출에 기여합니다. 학생들은 머신러닝, 딥러닝, 비쥬얼라이제이션, 최적화 등의 최신 기술을 활용하여 데이터를 분석하고 인사이트를 도출하는 실습 경험을 통해 실무 역량을 강화할 수 있습니다. (유사도: 10.09)
- 디지털경영전공은 다양한 배경을 가진 학생들이 모여 함께 성장하는 곳입니다. 전공과목은 프로그래밍과 데이터 분석에 필요한 기초 개념과 실습을 체계적으로 학습하도록 구성되어 있습니다. 때문에 문과 출신이더라도 열정과 노력으로 데이터 분석과 프로그래밍을 익힐 수 있습니다. 빅데이터와 인공지능의 기초를 익히고 기업 경영에 활용하는 방법을 익힌다면 여러분 앞에는 성공을 위한 무한한 가능성이 열릴 것입니다. 배우고자 하는 의지가 있고, 열정이 넘치는 모든 학생을 환영합니다. (유사도: 10.04)
질문: 기업에서 필요한 가치는 무엇인가요?
답변: 인사이트
참조 문장:
- 기업에서 필요한 가치는 인사이트를 통한 비즈니스 액션에 의해 창출됩니다. 빅데이터는 단순히 많이 모아 놓는다고, 기업에게 필요한 가치를 창출 할 수가 없습니다. 때문에 빅데이터로부터 패턴을 찾거나 의미를 찾아 비즈니스에 적용하는 과정이 필요합니다. 바로 이 빅데이터의 특정 패턴이나 의미에서 인사이트(insight)를 얻는 데이터 분석을 DATA ANALYST가 수행하게 됩니다 (유사도: 8.43)
- 빅데이터 인공지능과 경영학의 만남을 추구합니다. (유사도: 7.08)
- 디지털경영전공은 데이터 분석 능력과 비즈니스 마인드를 겸비한 실무 융합형 인재 양성을 목표로 하고 있습니다. 빅데이터와 인공지능 